In [35]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd
from flask import Flask, jsonify, request
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import pickle

In [36]:
cleaned_df = pd.read_csv('./cleaned_car_data.csv')
cleaned_df = cleaned_df.drop(['Name'], axis=1)
df_numeric = cleaned_df.dropna()

In [37]:
app = Flask(__name__)

X = df_numeric.drop('Price', axis=1)
y = df_numeric['Price']

imputer = SimpleImputer(strategy='mean', add_indicator=False)

try:
    with open('./Pickles/knn_model.pkl', 'rb') as file:
        knn_model = pickle.load(file)
        
except FileNotFoundError:

    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    knn_model = KNeighborsRegressor(n_neighbors=1, algorithm='auto', metric='minkowski', metric_params=None, leaf_size=30, p=2, n_jobs=None)
    knn_model.fit(X_imputed, y)
    
    with open('knn_model.pkl', 'wb') as file:
        pickle.dump(knn_model, file)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json(force=True)

        if 'features' in data and isinstance(data['features'], dict):
            features_dict = data['features']

            features_ordered = [features_dict.get(feature, np.nan) for feature in X.columns]
            features_df = pd.DataFrame([features_ordered])

            X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
            features_df_imputed = pd.DataFrame(imputer.transform(features_df), columns=features_df.columns)


            knn_prediction_normalized = knn_model.predict(features_df_imputed)
            knn_prediction_original = knn_prediction_normalized * (y.max() - y.min()) + y.min()
            
            
            return jsonify({'prediction': knn_prediction_original.tolist()})
        else:
            return jsonify({'error': 'Invalid or missing "features" data in the request'}), 400
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return jsonify({'error': 'Internal Server Error'}), 500
    
if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [09/Dec/2023 23:52:20] "POST /predict HTTP/1.1" 200 -
